# Group 1 Capstone Project Work
### Make sure to make / checkout your own branch before making changes

# Maddie Hince

In [177]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('zippedData/im.db')

In [178]:
ratings_df = pd.read_sql("""SELECT movie_ratings.movie_id, movie_ratings.averagerating, movie_ratings.numvotes, movie_basics.primary_title, movie_basics.start_year, movie_basics.genres 
                            FROM movie_ratings
                            JOIN movie_basics USING(movie_id)
                            WHERE movie_basics.start_year >= 2010""", conn)

In [189]:
ratings_df.dropna(how='any', axis=0, inplace=True)

In [190]:
sorted_by_avg = ratings_df.groupby('genres').mean().sort_values(by='averagerating', ascending=False)
# filter out genres with less than 62 votes
sorted_by_avg['numvotes'].describe()

count       923.000000
mean       4378.464914
std       13265.696554
min           5.000000
25%          62.250000
50%         322.200000
75%        2469.010861
max      181285.417323
Name: numvotes, dtype: float64

In [194]:
# 15 highest-rated movie genres with more than 62 total votes
sorted_by_avg[sorted_by_avg['numvotes'] >= 62].head(15)

,averagerating,numvotes,start_year
genres,,,
"Mystery,News,Thriller",8.400000,92.000000,2019.000000
"Documentary,Music,War",8.300000,119.625000,2015.000000
"Comedy,History,Musical",8.300000,172.000000,2016.000000
"Animation,Crime,Mystery",8.200000,526.000000,2014.000000
"Romance,Thriller,War",8.200000,165.000000,2017.000000
"Comedy,Musical,Western",8.200000,186.000000,2015.000000
"Animation,Documentary,Mystery",8.200000,1014.000000,2017.000000
"Documentary,Fantasy,Horror",8.100000,209.500000,2013.000000
"Animation,Crime,Documentary",8.066667,2198.000000,2013.000000


# Cat Murad

# Elijah Soba

# Luis Estrada 

# Kilmar Lazo 

# Michael Eugene